<a href="https://colab.research.google.com/github/kilingki/Quant_ML_Project/blob/master/KQR_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [54]:
# 5개의 분리된 엑셀 파일을 불러온다
input_ = []
s_time = time.time()
for i in range(5):
  input_.append(pd.read_excel('/gdrive/My Drive/input_'+str(i+1)+'.xlsx'))
print('Elapsed :', time.time() - s_time)

Elapsed : 42.34544134140015


In [55]:
# 전처리 1 : (연월종목 X 입력변수) 의 2차원 Dataframe 생성

# 5개 각각의 엑셀 파일을 2차원 dataframe으로 정리한 뒤 5개를 1개로 병합
num_date = 58

new_input = []
for input_data in input_:

  for i in range(1, num_date): # 날짜 -> string
    input_data['Code'][i] = input_data['Code'][i].strftime('%Y%m')[2:]

  # 날짜 리스트
  date = input_data['Code'].tolist()[1:]

  num_input = 28 # 입력 변수 갯수
  num_stock = (input_data.shape[1] - 1)//num_input # 한 엑셀 파일당 종목 갯수

  # 종목별 데이터 쪼개기
  input_data_list = []
  for i in range(num_stock):
    input_data_list.append(input_data.iloc[:, num_input*i+1 : num_input*(i+1)+1])
  
  # 입력 변수 리스트
  input_list = input_data_list[0].loc[0,:].tolist()

  # 종목 당 (연월종목 X 입력변수) Dataframe 구성
  for i in range(num_stock):
    # 종목 코드 추출
    stock_code = input_data_list[i].columns[0]

    # 날짜 + 종목의 새 Index 구성
    new_date = []
    
    for t in date:
      new_date.append(t + stock_code)

    # Column 설정
    input_data_list[i].columns = input_list
    input_data_list[i].drop([0], inplace=True)

    # Index 설정
    input_data_list[i].index = new_date
  
  # 종목 Dataframe 결합
  new_input.append(pd.concat(input_data_list))

# 엑셀 별 Dataframe 결합
all_input = pd.concat(new_input)
all_input

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,수정주가,월간평균거래량,월간수익률,20일거래량회전율,60일변동성(표준편차),20일평균대차거래체결량비중,월간평균 공매도수량/거래량,외국인지분율,유동주식비율(보통주),유동자산,비유동자산,부채총계,자본금,기타포괄이익누계액,비지배주주지분,Cash Earnings,영업활동으로인한현금흐름,투자활동으로인한현금흐름,CAPEX,배당금,자본의증감,현금의증가,매출원가,매출총이익,판매비와관리비,인건비및복리후생비,영업이익,당기순이익
1504A000020,8330,218978,10.92,0.007854,0.020248,0,0.72,6.36,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1505A000020,9370,332579,12.48,0.010915,0.029854,0,0.4,7,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1506A000020,9070,295146,-3.2,0.008875,0.033701,0,0.88,8.21,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1507A000020,8160,596820,-10.03,0.017638,0.040963,0,1.39,8.54,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1508A000020,7100,130112,-12.99,0.004658,0.036095,0.02,2.45,8.56,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908A950180,17350,62520,-7.71,0.003066,0.034115,0.01,0.57,78.99,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909A950180,17500,61914,0.86,0.002951,0.031125,0,0.36,79.35,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1910A950180,19000,60167,8.57,0.002937,0.028425,0,0.28,80.74,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1911A950180,16400,56079,-13.69,0.002624,0.020576,0,0.97,81.05,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# 전처리 2 : Dataset X, y 시간 조정
# 주식 종목 당 X(t) <-> y(t+1) 이 연결되도록 한다.
# -> X : 가장 최근 값 삭제 / y : 가장 과거 값 삭제

# 종목 종류 수
num_stock = all_input.shape[0]//num_date 

# y (Label) 분리
all_label = all_input[['월간수익률', '60일변동성(표준편차)']] 

# 종목 별 X 가장 최근 값 제거
input_idx_del = []
for i in range(num_stock):
  input_idx_del.append(all_input.index[num_date*(i+1) - 1])
all_input = all_input.drop(input_idx_del)

# 종목 별 y 가장 과거 값 제거
label_idx_del = []
for i in range(num_stock):
  label_idx_del.append(all_label.index[num_date * i])
all_label = all_label.drop(label_idx_del)

# X, y 결합
all_input['월간수익률'] = all_label['월간수익률'].tolist()
all_input['60일변동성(표준편차)'] = all_label['60일변동성(표준편차)'].tolist()
all_input

,수정주가,월간평균거래량,월간수익률,20일거래량회전율,60일변동성(표준편차),20일평균대차거래체결량비중,월간평균 공매도수량/거래량,외국인지분율,유동주식비율(보통주),유동자산,비유동자산,부채총계,자본금,기타포괄이익누계액,비지배주주지분,Cash Earnings,영업활동으로인한현금흐름,투자활동으로인한현금흐름,CAPEX,배당금,자본의증감,현금의증가,매출원가,매출총이익,판매비와관리비,인건비및복리후생비,영업이익,당기순이익
1504A000020,8330,218978,12.48,0.007854,0.029854,0,0.72,6.36,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1505A000020,9370,332579,-3.20,0.010915,0.033701,0,0.4,7,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1506A000020,9070,295146,-10.03,0.008875,0.040963,0,0.88,8.21,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1507A000020,8160,596820,-12.99,0.017638,0.036095,0,1.39,8.54,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1508A000020,7100,130112,-1.13,0.004658,0.031218,0.02,2.45,8.56,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908A950180,17350,62520,-7.71,0.003066,0.034115,0.01,0.57,78.99,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909A950180,17500,61914,0.86,0.002951,0.031125,0,0.36,79.35,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1910A950180,19000,60167,8.57,0.002937,0.028425,0,0.28,80.74,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1911A950180,16400,56079,-13.69,0.002624,0.020576,0,0.97,81.05,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br>
월단위 (9)<br>
<br>
수정주가<br>
월간평균거래량             <br>
월간수익률               <br>
20일거래량회전율          <br>
60일변동성(표준편차)         <br>
20일평균대차거래체결량비중    <br>
월간평균 공매도수량/거래량    <br>
외국인지분율              <br>
유동주식비율(보통주)          <br>
<br>
연단위 (18) : 12개월 동안 같은 값<br>
<br>
유동자산        <br>
비유동자산                <br>
부채총계                 <br>
자본금                   <br>
기타포괄이익누계액            <br>
Cash Earnings      <br>
영업활동으로인한현금흐름         <br>
투자활동으로인한현금흐름        <br>
CAPEX              <br>
배당금                <br>
자본의증감              <br>
현금의증가                <br>
매출원가              <br>
매출총이익               <br>
판매비와관리비              <br>
인건비및복리후생비          <br>
영업이익                 <br>
당기순이익                <br>

In [57]:
# 전처리 3 : 결측 값 제거
# y 값 (월간수익률, 표준편차)을 다른 값으로 채워넣기엔 성능 저하가 예측되기에 제거한다.

# (월간수익률, 표준편차) 이 NaN 인 Row 제거
all_input = all_input[pd.notnull(all_input['월간수익률'])]
all_input = all_input[pd.notnull(all_input['60일변동성(표준편차)'])]

# 종목 종류 수 재설정
num_stock = all_input.shape[0]//num_date 
all_input.isnull().sum() # Column 당 NaN 값 갯수

수정주가                141
월간평균거래량             136
월간수익률                 0
20일거래량회전율           280
60일변동성(표준편차)          0
20일평균대차거래체결량비중    15699
월간평균 공매도수량/거래량    25230
외국인지분율              149
유동주식비율(보통주)       13083
유동자산              16765
비유동자산             17181
부채총계              13892
자본금               13892
기타포괄이익누계액         29089
비지배주주지분           62500
Cash Earnings     14496
영업활동으로인한현금흐름      13901
투자활동으로인한현금흐름      14019
CAPEX             14978
배당금               14554
자본의증감             14139
현금의증가             13892
매출원가              19542
매출총이익             14639
판매비와관리비           13901
인건비및복리후생비         14494
영업이익              13754
당기순이익             13745
dtype: int64

In [58]:
# NaN 값이 너무 많은 Column 제거
all_input.drop(['비지배주주지분'], axis='columns', inplace=True)
num_input -= 1 # Input 수 재설정

# 2번째로 NaN 값이 제일 많은 Column (기타포괄이익누계액)이 NaN인 Row 제거 
all_input = all_input[pd.notnull(all_input['기타포괄이익누계액'])]

all_input.isnull().sum() # NaN 값이 특정 Row에 몰려 있음을 확인할 수 있다.

수정주가                63
월간평균거래량             59
월간수익률                0
20일거래량회전율          116
60일변동성(표준편차)         0
20일평균대차거래체결량비중    2292
월간평균 공매도수량/거래량    9080
외국인지분율              63
유동주식비율(보통주)          4
유동자산              2835
비유동자산             2835
부채총계                 0
자본금                  0
기타포괄이익누계액            0
Cash Earnings      113
영업활동으로인한현금흐름         0
투자활동으로인한현금흐름        56
CAPEX              450
배당금                611
자본의증감              211
현금의증가                0
매출원가              4691
매출총이익              682
판매비와관리비              0
인건비및복리후생비          429
영업이익                 0
당기순이익                0
dtype: int64

In [59]:
# 나머지 NaN 값을 해당 Column의 평균으로 채운다
# 모델 성능 저하 요소
for column in all_input.columns:
  all_input[column].fillna(all_input[column].mean(), inplace=True)
all_input.isnull().sum()

수정주가              0
월간평균거래량           0
월간수익률             0
20일거래량회전율         0
60일변동성(표준편차)      0
20일평균대차거래체결량비중    0
월간평균 공매도수량/거래량    0
외국인지분율            0
유동주식비율(보통주)       0
유동자산              0
비유동자산             0
부채총계              0
자본금               0
기타포괄이익누계액         0
Cash Earnings     0
영업활동으로인한현금흐름      0
투자활동으로인한현금흐름      0
CAPEX             0
배당금               0
자본의증감             0
현금의증가             0
매출원가              0
매출총이익             0
판매비와관리비           0
인건비및복리후생비         0
영업이익              0
당기순이익             0
dtype: int64

In [74]:
# 전처리 4: Data Normalization
# Column별로 -1 ~ 1 범위로 맞춰준다.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(all_input)
all_input_1 = pd.DataFrame(scaler.transform(all_input), columns=all_input.columns)
all_input_1.index = all_input.index
all_input_1

,수정주가,월간평균거래량,월간수익률,20일거래량회전율,60일변동성(표준편차),20일평균대차거래체결량비중,월간평균 공매도수량/거래량,외국인지분율,유동주식비율(보통주),유동자산,비유동자산,부채총계,자본금,기타포괄이익누계액,Cash Earnings,영업활동으로인한현금흐름,투자활동으로인한현금흐름,CAPEX,배당금,자본의증감,현금의증가,매출원가,매출총이익,판매비와관리비,인건비및복리후생비,영업이익,당기순이익
1504A000020,-0.990932,-0.997588,-0.709064,-0.988525,-0.843523,-1.000000,-0.968400,-0.865126,0.927871,-0.998520,-0.997957,-0.999661,-0.987137,-0.458033,-0.929103,-0.709872,0.846885,-0.999959,-0.999562,0.520543,-0.463730,-0.998282,-0.970392,-0.987176,-0.990737,-0.930199,-0.878108
1505A000020,-0.989781,-0.996337,-0.751147,-0.984052,-0.823359,-1.000000,-0.982445,-0.851553,0.927871,-0.998520,-0.997957,-0.999661,-0.987137,-0.458033,-0.929103,-0.709872,0.846885,-0.999959,-0.999562,0.520543,-0.463730,-0.998282,-0.970392,-0.987176,-0.990737,-0.930199,-0.878108
1506A000020,-0.990113,-0.996749,-0.769479,-0.987033,-0.785296,-1.000000,-0.961378,-0.825893,0.927871,-0.998520,-0.997957,-0.999661,-0.987137,-0.458033,-0.929103,-0.709872,0.846885,-0.999959,-0.999562,0.520543,-0.463730,-0.998282,-0.970392,-0.987176,-0.990737,-0.930199,-0.878108
1507A000020,-0.991120,-0.993427,-0.777423,-0.974229,-0.810811,-1.000000,-0.938995,-0.818895,0.927871,-0.998520,-0.997957,-0.999661,-0.987137,-0.458033,-0.929103,-0.709872,0.846885,-0.999959,-0.999562,0.520543,-0.463730,-0.998282,-0.970392,-0.987176,-0.990737,-0.930199,-0.878108
1508A000020,-0.992293,-0.998567,-0.745592,-0.993194,-0.836373,-0.993721,-0.892473,-0.818471,0.927871,-0.998520,-0.997957,-0.999661,-0.987137,-0.458033,-0.929103,-0.709872,0.846885,-0.999959,-0.999562,0.520543,-0.463730,-0.998282,-0.970392,-0.987176,-0.990737,-0.930199,-0.878108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1901A950170,-0.992270,-0.998583,-0.682627,-0.994668,-0.848167,-0.996860,-0.920562,-0.783480,0.875458,-0.997501,-0.998513,-0.999548,-0.977463,-0.458233,-0.929207,-0.710043,0.845050,-0.998810,-0.999012,0.536331,-0.457908,-0.997839,-0.965734,-0.978566,-0.988554,-0.929607,-0.878008
1902A950170,-0.990511,-0.992322,-0.736708,-0.974276,-0.859278,-1.000000,-0.987711,-0.798537,0.875458,-0.997501,-0.998513,-0.999548,-0.977463,-0.458233,-0.929207,-0.710043,0.845050,-0.998810,-0.999012,0.536331,-0.457908,-0.997839,-0.965734,-0.978566,-0.988554,-0.929607,-0.878008
1905A950180,-0.970001,-0.990634,-0.793607,-0.941008,-0.712094,-0.993721,-0.993856,0.702471,0.856424,-0.996642,-0.999827,-0.999934,-0.949828,-0.458259,-0.928718,-0.709541,0.846905,-0.999954,-0.999894,0.552842,-0.434270,-0.999715,-0.970669,-0.989214,-0.997333,-0.928811,-0.877025
1906A950180,-0.972656,-0.998250,-0.766204,-0.988533,-0.752998,-0.996860,-0.988150,0.701622,0.856424,-0.996642,-0.999827,-0.999934,-0.949828,-0.458259,-0.928718,-0.709541,0.846905,-0.999954,-0.999894,0.552842,-0.434270,-0.999715,-0.970669,-0.989214,-0.997333,-0.928811,-0.877025


In [0]:
# 전처리 4: Feature Analysis & Selection
# 모델 선택 1: Model Initialization & Cross Validation
# 모델 선택 2: Hyperparamater tunning
# 모델 선택 3: 
# Ensemble :
# Prediction :